Written by Zain Kamal [zain.eris.kamal@rutgers.edu](mailto:zain.eris.kamal@rutgers.edu) in 2024 March.

https://github.com/Humboldt-Penguin/redplanet

---
---
# Main 

In [1]:
"""
Written by Zain Kamal (zain.eris.kamal@rutgers.edu).
https://github.com/Humboldt-Penguin/redplanet

For more information, call `help(Crust)` or directly view docstring in `Crust/__init__.py`.

...

TODO:
    [ ] Type hinting
    [ ] Finish writing/fleshing out docstrings. 
    [ ] Add proper units + attributes + citations to xarray dataset. 
    [ ] Write a `plot` function? Need to rewrite the corresponding `visualize` function in `redplanet.utils` first (rename to `plot` there as well). 
    [ ] When using a coarsening factor for `load_topo`, also rechunk dask (or just remove this and leave the feature for people who want more control with get rawdata?)

"""



from redplanet import utils

from pathlib import Path
# import json

import pooch
import numpy as np
import xarray as xr
import rioxarray
import dask
import pyshtools as pysh
import pandas as pd



''' ————————————————————————————— Type Hinting ————————————————————————————— '''

from typing import Iterable, Union, Annotated, Literal, TypeVar, Any

NativeArray_1D_Numeric = Iterable[float]    # 1D lists, tuples, arrays

NumpyNumberType = TypeVar("NumpyNumberType", bound=np.number)
NumpyArray_1D_Numeric = Annotated[np.typing.NDArray[NumpyNumberType], Literal["N"]]

Array_1D_Numeric = Union[NativeArray_1D_Numeric, NumpyArray_1D_Numeric]







''' ——————————— Global Variables (Intentionally User-Accessible) ——————————— '''

## path where pooch downloads/caches data.
dirpath_data_root = pooch.os_cache('redplanet')













''' ######################################################################## '''
'''                             Load/Format Data                             '''
''' ######################################################################## '''





''' ——————————————————————————— Dichotomy Coords ——————————————————————————— '''


dat_dichotomy_coords : np.ndarray = ...  # Nx2 numpy array of dichotomy coordinates, structured (lon, lat).


def _init_dichotomy() -> None:

    global dat_dichotomy_coords

    ## lazy initialization
    if dat_dichotomy_coords is not ...:
        return
    else:
        pass  # needed to avoid pylance annoyances

    ## download / cache dichotomy coords
    with utils.disable_pooch_logger():
        fpath_dichotomy_coords = pooch.retrieve(
            fname      = 'dichotomy_coordinates-JAH-0-360.txt',
            url        = r'https://drive.google.com/file/d/17exPNRMKXGwa3daTEBN02llfdya6OZJY/view?usp=sharing',
            known_hash = 'sha256:42f2b9f32c9e9100ef4a9977171a54654c3bf25602555945405a93ca45ac6bb2',
            path       = dirpath_data_root / 'Crust' / 'dichotomy',
            downloader = utils.download_gdrive_file,
        )
    fpath_dichotomy_coords = Path(fpath_dichotomy_coords)

    ## load into Nx2 numpy array of dichotomy coordinates, structured (lon, lat).'''
    dat_dichotomy_coords = np.loadtxt(fpath_dichotomy_coords)

    ## fix the lons (convert from `0->360` to `0->180 U -180->0` and sort)
    dat_dichotomy_coords[:,0] = utils.plon2slon(dat_dichotomy_coords[:,0])
    dat_dichotomy_coords = dat_dichotomy_coords[np.argsort(dat_dichotomy_coords[:,0])]

    ## add wraparound coordinates for safety / convenience
    dat_dichotomy_coords = np.vstack((
        dat_dichotomy_coords, 
        [dat_dichotomy_coords[0,0]+360, dat_dichotomy_coords[0,1]], 
        [dat_dichotomy_coords[1,0]+360, dat_dichotomy_coords[1,1]], 
    ))

    return














''' ——————————————————— Topography (2 options, DEM or SH) —————————————————— '''


dat_dem_xr : xr.DataArray


def load_topo(
        model_type : str = 'DEM', 
        **kwargs   : Any, 
) -> None:
    
    match model_type.lower():

        case 'dem':
            _load_topo_dem(**kwargs)
        case 'sh':
            _load_topo_sh(**kwargs)
        case _:
            raise ValueError(f'Given argument `{model_type = }` is not valid. Please choose either "DEM" or "SH" (case insensitive).')









def _load_topo_dem(
    coarsen_factor   : int      = 1, 
    fpath_custom_DEM : str|Path = None, 
    chunk_size       : str      = '128MB', 
    quiet_download   : bool     = False, 
) -> None:
    """
    for `chunk_size` see https://docs.dask.org/en/latest/array-best-practices.html#select-a-good-chunk-size
    
    REFERENCES:
    ------------
        Mars MGS MOLA DEM 463m v2:
            > https://astrogeology.usgs.gov/search/map/Mars/GlobalSurveyor/MOLA/Mars_MGS_MOLA_DEM_mosaic_global_463m

    """

    global dat_dem_xr



    # ————————————————————————————————————————————————————————————————————
    '''import DEM (digital elevation model) to xarray'''

    ## download TIF (if no custom path provided)
    if quiet_download:
        downloader = utils.download_gdrive_file
    else:
        downloader = utils.download_gdrive_file_SHOWPROGRESS

    if fpath_custom_DEM is None:
        with utils.disable_pooch_logger():
            fpath_custom_DEM = pooch.retrieve(
                fname      = 'Mars_MGS_MOLA_DEM_mosaic_global_463m.tif',
                url        = r'https://drive.google.com/file/d/1ACMocVNzs7pFwxulLOp2vqQ24LjLOVuU/view?usp=sharing',
                known_hash = 'sha256:38a4eb0b4452855b8dabfac40a367b458555ab4c01b31235807ad0a53c031f4c',
                path       = dirpath_data_root / 'Crust' / 'topo',
                downloader = downloader,
            )
    

    ## load TIF     *(chunked with dask)*
    fpath_custom_DEM = Path(fpath_custom_DEM)

    if chunk_size != '128MB':
        dask.config.set({'array.chunk-size': chunk_size})

    dat_dem_xr = (
        rioxarray.open_rasterio(fpath_custom_DEM, chunks={'x': 'auto', 'y': 'auto'})
        .sel(band=1)
        .drop_vars(['band', 'spatial_ref'])
    )


    # ————————————————————————————————————————————————————————————————————
    '''post-processing'''
    
    ## transform coords
    R = 3396190.0   # mars radius per IAU 2000 definition
    standard_parallel = 0 
    scale = np.cos(np.radians(standard_parallel))
    dat_dem_xr['x'] = ((dat_dem_xr.x / (R * scale)) * (180 / np.pi)).data
    dat_dem_xr['y'] = ((dat_dem_xr.y / R) * (180 / np.pi)).data
    dat_dem_xr = dat_dem_xr.rename({'x': 'lon', 'y': 'lat'})
    dat_dem_xr = dat_dem_xr.sortby('lat')

    # ## convert elevation to topo
    # dat_dem_xr = (dat_dem_xr + R) * 1e-3
    ## ^ AVOID THIS, it significantly increases array size (2GB -> ~8GB).

    ## metadata
    dat_dem_xr.attrs = {
        'units': 'meters'
    }


    # ————————————————————————————————————————————————————————————————————
    '''optional: downsample for speed/storage'''
    if coarsen_factor > 1:
        dat_dem_xr = dat_dem_xr.coarsen(
            lon=coarsen_factor, 
            lat=coarsen_factor, 
            boundary='trim'
        ).mean()


    return













def _load_topo_sh(
    grid_spacing   : float = 0.1,
    quiet_download : bool  = False,
) -> None:
    """
    for chunk_size see https://docs.dask.org/en/latest/array-best-practices.html#select-a-good-chunk-size
    

    
    REFERENCES:
    ------------
        Mars MGS MOLA DEM 463m v2:
            > https://astrogeology.usgs.gov/search/map/Mars/GlobalSurveyor/MOLA/Mars_MGS_MOLA_DEM_mosaic_global_463m

    """
    global dat_dem_xr


    

    ''' ———————————————————————————————————————————————————————————————————— '''
    '''arg handling'''
    

















''' ——————————————————————— Mohorovičić Discontinuity —————————————————————— '''



dat_moho_xr : xr.DataArray


def load_moho(
    RIM                  : str, 
    insight_thickness    : int, 
    rho_north            : int, 
    rho_south            : int, 
    suppress_model_error : bool = False,
) -> bool:
    """
    NOTE: All data and the following spreadsheet is taken *directly* from the following work with no alterations:
        - Wieczorek, M. A. (2022). InSight Crustal Thickness Archive [Data set]. Zenodo. https://doi.org/10.5281/zenodo.6477509
    
    Summary of all available models (reuploaded to Google Sheets with for convenience — no alterations were made):
        - https://docs.google.com/spreadsheets/d/1ZDILcSPdbXAFp60VfyC4xTZzdnAVhx_U/edit?usp=sharing&ouid=107564547097010500390&rtpof=true&sd=true

    You can also view all available models with `Crust.get_moho_registry()`.
    """

    global dat_moho_xr

    _init_moho_shcoeffs_registry()

    model_name = f'{RIM}-{insight_thickness}-{rho_south}-{rho_north}'


    # ————————————————————————————————————————————————————————————————————
    '''download/fetch file containing SH coefficients for the chosen model'''

    try:
        model_metadata = _df_moho_shcoeffs_registry.loc[model_name].to_dict()
    except KeyError:
        if suppress_model_error:
            return False
        else:
            raise ValueError(f'No Moho model with the inputs {model_name} exists.')

    with utils.disable_pooch_logger():
        fpath_moho_shcoeffs = pooch.retrieve(
            fname      = f'{model_name}.txt', 
            url        = model_metadata['link'], 
            known_hash = model_metadata['hash'], 
            path       = dirpath_data_root / 'Crust' / 'moho' / 'SH_coeffs', 
            downloader = utils.download_gdrive_file, 
        )
    fpath_moho_shcoeffs = Path(fpath_moho_shcoeffs)


    # ————————————————————————————————————————————————————————————————————
    '''load to xarray'''

    ## convert shcoeffs -> shgrid -> xarray
    dat_moho_xr = pysh.SHCoeffs.from_file(
        fname = fpath_moho_shcoeffs, 
        name  = model_name, 
    ).expand(
        grid   = 'DH2', 
        extend = True, 
    ).to_xarray()
    
    ## post-processing
    # dat_moho_xr = dat_moho_xr * 1e-3    # convert m -> km
    dat_moho_xr = utils.fix_pyshtools_coords(dat_moho_xr)
    dat_moho_xr.attrs.update({
        'units'                       : 'meters',
        'moho_model_name'             : model_name,
        'moho_model_RIM'              : RIM,
        'moho_model_insight_thickness': insight_thickness,
        'moho_model_rho_north'        : rho_north,
        'moho_model_rho_south'        : rho_south,
    })

    return True











_df_moho_shcoeffs_registry : pd.DataFrame = ...   # columns: 'link', 'hash', 'model_name', 'RIM', 'insight_thickness', 'rho_south', 'rho_north'. 


def _init_moho_shcoeffs_registry() -> None:

    global _df_moho_shcoeffs_registry

    ## lazy initialization
    if _df_moho_shcoeffs_registry is not ...:
        return
    else:
        pass   # needed to avoid pylance annoyances


    ## download a pre-computed registry of moho models, which provides a google drive download link and a sha256 hash for a given model name
    with utils.disable_pooch_logger():
        fpath_moho_shcoeffs_registry = pooch.retrieve(
            fname      = 'mohoSHcoeffs_rawdata_registry.json',
            url        = r'https://drive.google.com/file/d/17JJuTFKkHh651-rt2J2eFKnxiki0w4ue/view?usp=sharing',
            known_hash = 'sha256:1800ee2883dc6bcc82bd34eb2eebced5b59fbe6c593cbc4e9122271fd01c1491',
            path       = dirpath_data_root / 'Crust' / 'moho', 
            downloader = utils.download_gdrive_file,
        )
    fpath_moho_shcoeffs_registry = Path(fpath_moho_shcoeffs_registry)


    ## load to pandas dataframe and split 'model_name' into components
    _df_moho_shcoeffs_registry = (
        pd.read_json(
            fpath_moho_shcoeffs_registry, 
            orient='index', 
        ).reset_index(
        ).rename(
            columns={'index': 'model_name'}, 
        )
    )

    _df_moho_shcoeffs_registry[['RIM', 'insight_thickness', 'rho_south', 'rho_north']] = (
        _df_moho_shcoeffs_registry['model_name'].str.split('-', expand=True)
    )

    _df_moho_shcoeffs_registry['insight_thickness'] = _df_moho_shcoeffs_registry['insight_thickness'].astype(int)
    _df_moho_shcoeffs_registry['rho_south']         = _df_moho_shcoeffs_registry['rho_south']        .astype(int)
    _df_moho_shcoeffs_registry['rho_north']         = _df_moho_shcoeffs_registry['rho_north']        .astype(int)

    _df_moho_shcoeffs_registry.set_index('model_name', inplace=True)

    return    


















''' ######################################################################## '''
'''                                 Accessors                                '''
''' ######################################################################## '''




''' ——————————————————————————————— Dichotomy —————————————————————————————— '''

def is_above_dichotomy(lon: float, lat: float) -> None:
    ## Our approach is to find two closest points by longitude and linearly interpolate to find latitude (additional latitude checks and cross product are both slower)

    _init_dichotomy()   # consider duplicating function so this can be better vectorized?

    i_lon = np.searchsorted(
        dat_dichotomy_coords[:,0], 
        lon, 
        side='right'
    ) - 1
    
    llon, llat = dat_dichotomy_coords[i_lon]
    rlon, rlat = dat_dichotomy_coords[i_lon+1]

    tlat = llat + (rlat-llat)*( (lon-llon)/(rlon-llon) )
    return lat >= tlat

    # v1 = (rlon-llon, rlat-llat)
    # v2 = (rlon-lon, rlat-lat)
    # xp = v1[0]*v2[1] - v1[1]*v2[0]  # cross product magnitude
    # return xp >= 0










''' ————————————————————————————————— Moho ————————————————————————————————— '''


def get_moho_registry() -> pd.DataFrame: 
    """
    TODO: docstring
    """
    _init_moho_shcoeffs_registry()
    return _df_moho_shcoeffs_registry.copy()





def get_model_info() -> dict:
    attrs = dat_moho_xr.attrs
    return {
        'name'              : attrs.get('moho_model_name'), 
        'RIM'               : attrs.get('moho_model_RIM'), 
        'insight_thickness' : attrs.get('moho_model_insight_thickness'), 
        'rho_north'         : attrs.get('moho_model_rho_north'), 
        'rho_south'         : attrs.get('moho_model_rho_south'), 
    }




---
---
# New topo

## timer template

In [ ]:
import time

In [ ]:
# t0 = time.time()



# t1 = time.time()
# print(f'{t1-t0}')   # seconds

## [1] time: gdrive download

ANS: 12.210543394088745

In [ ]:
t0 = time.time()




with utils.disable_pooch_logger():
    fpath_Mars_shape_5759 = pooch.retrieve(
        fname      = 'Mars_shape_5759.sh',
        url        = r'https://drive.google.com/file/d/1jW3xkcjIONY3SKCU-NCzvYqvOVppj3zx/view?usp=drive_link',
        known_hash = 'sha256:b5f129b3e26669ca0f977a1d486fdd3de3796c555c90c5dc5a6c51a9c81e9bc0',
        path       = dirpath_data_root / 'Crust' / 'topo',
        downloader = utils.download_gdrive_file_SHOWPROGRESS,
        # processor  = pooch.Decompress(),
    )




t1 = time.time()
print(f'{t1-t0}')   # seconds

## [2] time: load already decompressed .sh file (from gdrive)

In [ ]:
lmax_maxval = 5759
grid_spacing_minval = 180. / (2 * lmax_maxval + 2)
print(f'{grid_spacing_minval = }')

In [ ]:
t0 = time.time()


grid_spacing = 0.05

lmax = round(90. / grid_spacing - 1)
# grid_spacing = 180. / (2 * lmax + 2)


## convert shcoeffs -> shgrid -> xarray, and fix coords
dat_topo_xr = (
    pysh.SHCoeffs.from_file(
        fname  = fpath_Mars_shape_5759, 
        format = 'bshc', 
        lmax   = lmax, 
        name   = 'Mars_shape_5759', 
        units  = 'm', 
    ).expand(
        grid   = 'DH2', 
        extend = True, 
    ).to_xarray()
)

dat_topo_xr = utils.fix_pyshtools_coords(dat_topo_xr)


t1 = time.time()
print(f'{t1-t0}')   # seconds

In [ ]:
dat_topo_xr

In [ ]:
max_sh = dat_topo_xr.max().item()
min_sh = dat_topo_xr.min().item()

print(f'{max_sh = :.2f}')
print(f'{min_sh = :.2f}')
print(f'range = {max_sh - min_sh :.2f}')

In [ ]:
dat_topo_xr.sel(
    lon    = np.arange(-180,180,1), 
    lat    = np.arange(-90,90,1), 
    method = 'nearest', 
).plot(figsize=(10,5))

In [ ]:
dat_topo_xr.sel(
    lon    = slice(170,179.9), 
    lat    = slice(-20,-10), 
).plot()

In [ ]:
help(_load_topo_dem)
# _load_topo_dem()
# dat_dem_xr

---
## other shit

In [ ]:
grid_spacing = 0.1

lmax = round(90. / grid_spacing - 1)
grid_spacing = 180. / (2 * lmax + 2)

lmax

In [ ]:
# help(pysh.SHCoeffs.from_file)

---
## ~~downsampling?~~

- naur, it's not `rioxarray.open_rasterio` that takes time, it's the fucking hash check lmfao
    - try xxhash?

In [ ]:
# # if custom_DEM_fpath is None:
# with utils.disable_pooch_logger():
#     custom_DEM_fpath = pooch.retrieve(
#         fname      = 'Mars_MGS_MOLA_DEM_mosaic_global_463m.tif', 
#         url        = r'https://drive.google.com/file/d/1ACMocVNzs7pFwxulLOp2vqQ24LjLOVuU/view?usp=sharing', 
#         # known_hash = 'sha256:38a4eb0b4452855b8dabfac40a367b458555ab4c01b31235807ad0a53c031f4c', 
#         known_hash = 'xxh3_64:a0dc027e687f855f', 
#         # known_hash = None, 
#         path       = dirpath_data_root / 'Crust' / 'topo', 
#         downloader = utils.download_gdrive_file, 
#     )

# ## load TIF     *(chunked with dask)*
# custom_DEM_fpath = Path(custom_DEM_fpath)

In [ ]:
# pooch.file_hash(custom_DEM_fpath, alg='xxh32')
# pooch.file_hash(custom_DEM_fpath, alg='xxh64')
# pooch.file_hash(custom_DEM_fpath, alg='xxh128')

# pooch.file_hash(custom_DEM_fpath, alg='xxh3_64')
# pooch.file_hash(custom_DEM_fpath, alg='xxh3_128')

In [ ]:
'''load tif with downsample (ULTIMATELY USELESS BC CHUNKING WITH DASK! SPEEDUP COMES FROM BETTER HASHING ALG)'''
pass

# import rasterio

# # Define the downsampling factor
# downsampling_factor = 2  # This means each 2x2 block of pixels becomes one pixel

# with rasterio.open(custom_DEM_fpath) as dataset:
#     # Calculate new dimensions
#     new_height = dataset.height // downsampling_factor
#     new_width = dataset.width // downsampling_factor

#     # Read and downsample the data
#     data = dataset.read(
#         1, 
#         out_shape  = (1, new_height, new_width), 
#         resampling = rasterio.enums.Resampling.bilinear, 
#     )

    
# # if chunk_size != '128MB':
# #     dask.config.set({'array.chunk-size': chunk_size})

# dat_dem_xr = (
#     rioxarray.open_rasterio(data, chunks={'x': 'auto', 'y': 'auto'})
#     .sel(band=1)
#     .drop_vars(['band', 'spatial_ref'])
# )

---
## figuring out proper elevation correction for DEM by comparing with SH model

In [ ]:
'''use this to get a better crs_wkt!'''

# custom_DEM_HIRES_fpath = Path(r'C:\Users\Eris\AppData\Local\redplanet\redplanet\Cache\Crust\topo\Mars_HRSC_MOLA_BlendDEM_Global_200mp_v2.tif')

# dat_dem_xr = (
#     rioxarray.open_rasterio(
#         custom_DEM_HIRES_fpath, 
#         # fpath_custom_DEM, 
#         chunks={'x': 'auto', 'y': 'auto'}, 
#     )
#     # .sel(band=1).drop_vars(['band', 'spatial_ref'])   # MUST remove this line!!!
# )

# print(dat_dem_xr.spatial_ref.GeoTransform)
# print()
# print(dat_dem_xr.spatial_ref.crs_wkt)

'''     ^^^     ^^^     ^^^



outputs for the better hi-res 200m dem end up being::

    -180.0 0.003374120830641 0.0 90.0 0.0 -0.003374120830641

    GEOGCS["GCS_Mars_2000_Sphere",DATUM["Mars_2000_(Sphere)",SPHEROID["Mars_2000_Sphere_IAU_IAG",3396190,0]],PRIMEM["Reference_Meridian",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST]]

    
for 463m dem:

    -10669675.197320545 463.0935415503709 0.0 5334837.598660273 0.0 -463.0935415503709

    PROJCS["Equirectangular Mars",GEOGCS["GCS_Mars",DATUM["D_Mars",SPHEROID["Mars_localRadius",3396190,0]],PRIMEM["Reference_Meridian",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Equirectangular"],PARAMETER["standard_parallel_1",0],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


'''

pass

In [2]:
# if custom_DEM_fpath is None:
with utils.disable_pooch_logger():
    fpath_custom_DEM = pooch.retrieve(
        fname      = 'Mars_MGS_MOLA_DEM_mosaic_global_463m.tif', 
        url        = r'https://drive.google.com/file/d/1ACMocVNzs7pFwxulLOp2vqQ24LjLOVuU/view?usp=sharing', 
        # known_hash = 'sha256:38a4eb0b4452855b8dabfac40a367b458555ab4c01b31235807ad0a53c031f4c', 
        # known_hash = 'xxh3_64:a0dc027e687f855f', 
        known_hash = None, 
        path       = dirpath_data_root / 'Crust' / 'topo', 
        downloader = utils.download_gdrive_file, 
    )

## load TIF     *(chunked with dask)*
fpath_custom_DEM = Path(fpath_custom_DEM)

In [ ]:
'''PROJECTION 1: WARPED VRT'''
'''THIS TURNS OUT TO BE SLOW AS FUCK WHEN ACCESSING'''
pass

# import rasterio

# target_crs_wkt = 'GEOGCS["GCS_Mars_2000_Sphere",DATUM["Mars_2000_(Sphere)",SPHEROID["Mars_2000_Sphere_IAU_IAG",3396190,0]],PRIMEM["Reference_Meridian",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST]]'

# # vrt = rasterio.vrt.WarpedVRT(rasterio.open(custom_DEM_fpath), crs=target_crs_wkt)
# dat_dem_xr_v1 = (
#     rioxarray.open_rasterio(
#         filename = rasterio.vrt.WarpedVRT(
#             src_dataset = rasterio.open(fpath_custom_DEM), 
#             crs         = target_crs_wkt, 
#         ), 
#         chunks = {'x': 'auto', 'y': 'auto'}, 
#     )
#     .sel(band=1).drop_vars(['band', 'spatial_ref'])
#     .rename({'x': 'lon', 'y': 'lat'})
#     .sortby('lat', ascending=True)
# )


# dat_dem_xr_v1

In [4]:
'''PROJECTION 2: REPROJECTION'''
'''FAST AS FUCK, IDENTICAL TO PROJECTION 1 (WARPED VRT) BUT INITIAL REPROJ TAKES 45 SEC -- EITHER FIND A WAY TO SPEEDUP WITH DASK, OR JUST PRE-COMPUTE'''

target_crs_wkt = 'GEOGCS["GCS_Mars_2000_Sphere",DATUM["Mars_2000_(Sphere)",SPHEROID["Mars_2000_Sphere_IAU_IAG",3396190,0]],PRIMEM["Reference_Meridian",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST]]'


dat_dem_xr_v2 = (
    rioxarray.open_rasterio(
        filename = fpath_custom_DEM, 
        chunks   = {'x': 'auto', 'y': 'auto'}, 
    )
    .rio.reproject(target_crs_wkt)      ## this works, but it takes a while (~45 sec?) and you can't drop the spatial_ref attributes as i did earlier.
    .chunk({'x': 'auto', 'y': 'auto'})
    .sel(band=1).drop_vars(['band'])
    .rename({'x': 'lon', 'y': 'lat'})
    .sortby('lat', ascending=True)
    .chunk({'lon': 'auto', 'lat': 'auto'})
)


dat_dem_xr_v2.attrs = {
    'units'               : 'meters', 
    'data_source'         : 'https://astrogeology.usgs.gov/search/map/Mars/GlobalSurveyor/MOLA/Mars_MGS_MOLA_DEM_mosaic_global_463m', 
    'this_implementation' : 'https://github.com/Humboldt-Penguin/redplanet/blob/main/src/redplanet/Crust/Crust.py', 
}


print(f'{dat_dem_xr_v2.rio.crs = }\n\n')
dat_dem_xr_v2

dat_dem_xr_v2.rio.crs = CRS.from_wkt('GEOGCS["GCS_Mars_2000_Sphere",DATUM["Mars_2000_(Sphere)",SPHEROID["Mars_2000_Sphere_IAU_IAG",3396190,0]],PRIMEM["Reference_Meridian",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST]]')




<xarray.DataArray (lat: 23041, lon: 46081)> Size: 2GB
dask.array<rechunk-merge, shape=(23041, 46081), dtype=int16, chunksize=(8192, 8192), chunktype=numpy.ndarray>
Coordinates:
  * lon          (lon) float64 369kB -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * lat          (lat) float64 184kB -89.99 -89.99 -89.98 ... 89.98 89.99 90.0
    spatial_ref  int32 4B 0
Attributes:
    units:                meters
    data_source:          https://astrogeology.usgs.gov/search/map/Mars/Globa...
    this_implementation:  https://github.com/Humboldt-Penguin/redplanet/blob/...

In [8]:
dat_dem_xr_v2.to_zarr(
    store = Path.cwd() / 'Mars_MGS_MOLA_DEM_mosaic_global_463m_REPROJECTED.zarr'
)

In [10]:
dat_dem_xr_v2.to_netcdf(
    path = Path.cwd() / 'Mars_MGS_MOLA_DEM_mosaic_global_463m_REPROJECTED.nc'
)

In [ ]:
# '''PROJECTION 3: REPROJECTION W DASK'''


# dat_dem_xr_v3 = (
#     rioxarray.open_rasterio(
#         filename = fpath_custom_DEM, 
#         chunks   = {'x': 'auto', 'y': 'auto'}, 
#     )
#     .sel(band=1).drop_vars(['band'])
#     .sortby('y', ascending=True)
# )

# print(dat_dem_xr_v3.rio.crs)




# import rasterio
# target_crs_wkt = rasterio.crs.CRS.from_string('GEOGCS["GCS_Mars_2000_Sphere",DATUM["Mars_2000_(Sphere)",SPHEROID["Mars_2000_Sphere_IAU_IAG",3396190,0]],PRIMEM["Reference_Meridian",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST]]')


# from odc.geo import xr

# dat_dem_xr_v3 = (
#     dat_dem_xr_v3.odc.reproject(
#         how           = target_crs_wkt,
#         # resampling    = 'bilinear',
#         # resolution    = 'fit',
#         # dst_nodata    = -9999,
#         # num_threads = 16,
#     )
# )

# print()
# print(dat_dem_xr_v3.rio.crs)





# dat_dem_xr_v3




# # CRS.from_wkt('GEOGCS["GCS_Mars_2000_Sphere",DATUM["Mars_2000_(Sphere)",SPHEROID["Mars_2000_Sphere_IAU_IAG",3396190,0]],PRIMEM["Reference_Meridian",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST]]')
# # CRS.from_wkt('PROJCS["Equirectangular Mars",GEOGCS["GCS_Mars",DATUM["D_Mars",SPHEROID["Mars_localRadius",3396190,0]],PRIMEM["Reference_Meridian",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Equirectangular"],PARAMETER["standard_parallel_1",0],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]')




# # rasterio.crs.CRS

In [ ]:
'''for speed comparison with virtual warped projecting thing:::'''
'''somehow we're short one number and values are a little different, so you can't just change the coords lol.'''

dat_dem_xr_v0 = (
    rioxarray.open_rasterio(
        filename = fpath_custom_DEM, 
        chunks   = {'x': 'auto', 'y': 'auto'}, 
    )
    .sel(band=1).drop_vars(['band', 'spatial_ref'])
    .sortby('y')
)



# # ————————————————————————————————————————————————————————————————————
# '''post-processing'''

# ## transform coords
# R = 3396190.0   # mars radius per IAU 2000 definition
# standard_parallel = 0 
# scale = np.cos(np.radians(standard_parallel))
# dat_dem_xr['x'] = ((dat_dem_xr.x / (R * scale)) * (180 / np.pi)).data
# dat_dem_xr['y'] = ((dat_dem_xr.y / R) * (180 / np.pi)).data
# dat_dem_xr = dat_dem_xr.rename({'x': 'lon', 'y': 'lat'})
# dat_dem_xr = dat_dem_xr.sortby('lat')



dat_dem_xr_v0

In [ ]:
import time

t0 = time.time()

v0 = dat_dem_xr_v0.isel(
    x    = slice(1,1001), 
    y    = slice(1,1001), 
    # method = 'nearest', 
).values

t1 = time.time()

v2 = dat_dem_xr_v2.isel(
    lon    = slice(0,1000), 
    lat    = slice(0,1000), 
    # method = 'nearest', 
).values

t2 = time.time()

print(f'v0 took {t1-t0}')
print(f'v2 took {t2-t1}')

In [ ]:
# np.allclose(dat_dem_xr_v1.lat.values, dat_dem_xr_v2.lat.values)
np.allclose(v0, v2)

In [ ]:
import time

t0 = time.time()

v1 = dat_dem_xr_v1.sel(
    lon    = np.arange(-180,180,1), 
    lat    = np.arange(-90,90,1), 
    method = 'nearest', 
).values

t1 = time.time()

v2 = dat_dem_xr_v2.sel(
    lon    = np.arange(-180,180,1), 
    lat    = np.arange(-90,90,1), 
    method = 'nearest', 
).values

t2 = time.time()

print(f'v1 took {t1-t0}')
print(f'v2 took {t2-t1}')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(12,10))
plt.imshow(v2)
# plt.colorbar()

In [ ]:
np.allclose(v1, v2)

In [ ]:
plt.figure(figsize=(12,10))
plt.imshow(v2 - v1)
plt.colorbar()

In [ ]:
dat_dem_xr.sel(
    lon    = np.arange(-180,180,10), 
    lat    = np.arange(-90,90,10), 
    method = 'nearest', 
).plot(figsize=(10,5))

In [ ]:
dat_dem_xr_fixed.sel(
    lon    = np.arange(-180,180,10), 
    lat    = np.arange(-90,90,10), 
    method = 'nearest', 
).plot(figsize=(10,5))

In [ ]:
dat_dem_xr.sel(
    lon    = slice(170,179.9), 
    lat    = slice(-20,-10), 
).plot()

In [ ]:
max_dem = dat_dem_xr.max().values.item()
min_dem = dat_dem_xr.min().values.item()

print(f'{max_dem = :.2f}')
print(f'{min_dem = :.2f}')
print(f'range = {max_dem - min_dem :.2f}')



```raw
SUMMARY:

    sh vals:

        max_sh = 3417329.26
        min_sh = 3372922.99
        range  =   44406.26


    dem vals:

        max_dem = 21241.00
        min_dem = -8201.00
        range   = 29442.00
```

---
try choosing coords?

In [ ]:
lons = np.linspace(-180,180,100)
lats = np.linspace(-90,90,100)

# grid_sh = dat_topo_xr.interp(
grid_sh = dat_topo_xr.sel(
    lon    = lons, 
    lat    = lats, 
    # method = 'linear', 
    method = 'nearest', 
).values

# grid_dem = dat_dem_xr.interp(
grid_dem = dat_dem_xr.sel(
    lon    = lons, 
    lat    = lats, 
    method = 'nearest', 
).values

diffs = grid_sh - grid_dem

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(diffs)
plt.colorbar()

In [ ]:
# # diffs = []

# # for i in range(len(lons)):
# #     zero_sh = dat_topo_xr.sel(
# #         lon    = lons[i], 
# #         lat    = lats[i], 
# #         method = 'nearest'
# #     ).values.item()

# #     zero_dem = dat_dem_xr.sel(
# #         lon    = lons[i], 
# #         lat    = lats[i], 
# #         method = 'nearest'
# #     ).values.item()

# #     diffs.append(zero_sh - zero_dem)

# import matplotlib.pyplot as plt

# plt.hist(diffs, bins=100)

---
---
# `get`

In [ ]:
load_topo()
dat_dem_xr

In [ ]:
R = 3_396_190
dat_dem_xr + R

In [ ]:
load_moho(
    RIM               = 'Khan2022',
    insight_thickness = 39,
    rho_north         = 2900,
    rho_south         = 2900,
)

dat_moho_xr

In [ ]:
def get(
    quantity    : str, 
    lon         : float | Array_1D_Numeric, 
    lat         : float | Array_1D_Numeric, 
    interpolate : bool = False, 
    as_xarray   : bool = False, 
) -> float | Array_1D_Numeric | xr.DataArray:

    # ————————————————————————————————————————————————————————————————————
    '''checks'''

    if (np.any(lon < -180) or np.any(360 < lon)):
        raise ValueError(f'Longitude values must be in range [-180, 360].')
    if (np.any(lat < -90) or np.any(90 < lat)):
        raise ValueError(f'Latitude values must be in range [-90, 90].')
    lon = utils.plon2slon(lon) # this only modifies values btwn 180 and 360

    ## annoying floating point precision issues with numpy arange/linspace
    lon = np.round(lon, 10)
    lat = np.round(lat, 10)



    # ————————————————————————————————————————————————————————————————————
    '''accessing'''
    if interpolate:
        method = 'linear'
    else:
        method = 'nearest'

    match quantity.lower():

        case 'topo':
            result = dat_dem_xr.interp(lon=lon, lat=lat, assume_sorted=True, method=method)

        case 'moho':
            _init_moho_shcoeffs_registry()
            result = dat_moho_xr.interp(lon=lon, lat=lat, assume_sorted=True, method=method)
        
        case 'crust' | 'crustal thickness' | 'crthick':
            topo  = dat_dem_xr .interp(lon=lon, lat=lat, assume_sorted=True, method=method)
            moho  = dat_moho_xr.interp(lon=lon, lat=lat, assume_sorted=True, method=method)
            result = (topo-moho)
        
        case 'rho' | 'density' | 'crustal density':
            vec_is_above_dichotomy = np.vectorize(is_above_dichotomy)
            result_mask = vec_is_above_dichotomy(
                np.meshgrid(lon, lat)[0], 
                np.meshgrid(lon, lat)[1],
            )
            result = np.where(
                result_mask, 
                get_model_info()['rho_north'], 
                get_model_info()['rho_south'],
            )
        
        case _:
            raise Exception('Invalid quantity. Options are ["topo", "moho", "crust"/"crustal thickness"/"crthick", "rho"/"density"/"crustal density"].')

    return result






---
---
---
# scrap

In [ ]:
'''
NOTE: for "power-users" who want to work with the xarrays directly, instead of forcing them to use `get_rawdata` (inconvenient, verbose), just let them access the global variables directly (also allows them to customize the data as they want and still use my built-in functions). 
'''
# def get_rawdata(how=None):
#     """
#     `format` options: ['xarray', 'dict', 'dichotomy']

#     Note: when viewing/exploring dictionaries, it may help to call:
#         ```
#         from redplanet import utils
#         utils.print_dict(dat_something_dict)     # insert any dictionary here
#         ```
#     """
#     if how is None:
#         raise ValueError('Options are ["xarray", "dict", "dichotomy"].')

#     _initialize()
#     match how:
#         case 'xarray':
#             return dat_moho_xr
#         case 'dict':
#             return dat_crust_dict
#         case 'dichotomy':
#             return dat_dichotomy_coords
#         case _:
#             raise ValueError('Options are ["xarray", "dict", "dichotomy"].')






# def _update_dict_to_match_xrds():
#     global dat_crust_dict
#     dat_crust_dict = {
#         'lats': dat_moho_xr.lat.values,
#         'lons': dat_moho_xr.lon.values,
#         'attrs': dat_moho_xr.attrs, 
#     }
#     for data_var in list(dat_moho_xr.data_vars):
#         dat_crust_dict[data_var] = dat_moho_xr[data_var].values




In [ ]:
# def get(
#     quantity, 
#     lon          = None, 
#     lat          = None, 
#     lon_bounds   = None, ## i'm thinking of getting rid of `lon_bounds`/`lat_bounds`/`grid_spacing`/`num_points` option, and instead just having `lon`/`lat` be a float or a list/tuple/np.ndarray of floats (i.e. 1D). reason being, it's better to encourage the user to define their own `lons`/`lats` arrays using numpy so they can use the same input range across various different functions. this would also make it MUCH easier to combine into a single "get" function. but at that point am i just rewriting xarray's "sel" method??? i'm losing the chance to write more code, but i think i'd rather just stick with xarray since it natively supports stuff like `slice`
#     lat_bounds   = None, 
#     grid_spacing = None, 
#     num_points   = None, 
#     interpolate  = False, 
#     as_xarray    = False, 
# ):
#### WAIT but i need to check the coords and handle wraparound rip

















# def get_pt(
#     quantity, 
#     lon, 
#     lat, 
#     interpolate = False,
# ):

#     '''checks'''
#     _initialize()

#     if not (-180 <= lon <= 360):
#         raise ValueError(f'Given longitude coordinate {lon=} is out of range [-180, 360].')
#     if not (-90 <= lat <= 90):
#         raise ValueError(f'Given latitude coordinate {lat=} is out of range [-90, 90].')
    
#     lon = utils.plon2slon(lon) # this only modifies values btwn 180 and 360



#     '''accessing'''
#     if interpolate:
#         method = 'linear'
#     else:
#         method = 'nearest'

#     match quantity:

#         case 'topo' | 'moho':
#             val = dat_moho_xr.interp(lon=lon, lat=lat, assume_sorted=True, method=method)[quantity].item()
        
#         case 'crust' | 'crustal thickness' | 'crthick':
#             interped = dat_moho_xr.interp(lon=lon, lat=lat, assume_sorted=True, method=method)
#             val = (interped.topo - interped.moho).item()
        
#         case 'rho' | 'density' | 'crustal density':
#             if is_above_dichotomy(lon, lat):
#                 val = get_model_info()['rho_north']
#             else:
#                 val = get_model_info()['rho_south']
        
#         case _:
#             raise Exception('Invalid quantity. Options are ["topo", "moho", "crust"/"crustal thickness"/"crthick", "rho"/"density"/"crustal desntiy"].')

#     return val












# def get_region(
#     quantity, 
#     lons         = None,
#     lats         = None,
#     lon_bounds   = None, 
#     lat_bounds   = None, 
#     grid_spacing = None,
#     num_points   = None, 
#     interpolate  = False,
#     as_xarray    = False,
# ):
#     """
#     lon_bounds, clon_bounds, lat_bounds : tuple(float, float)
#         - Bounding box for data swath. 

#     grid_spacing : float
#         - Spacing between points being sampled in degrees. Note that original data is 5x5 degree bins.

#     """


#     '''args     (this approach is a bit verbose, but easy to understand and comprehensive)'''

#     error_msg = 'Invalid inputs for `get_region`. Options are: [1] `lons=..., lats=...` OR [2] `lon_bounds=..., lat_bounds=..., grid_spacing=...` OR [3] `lon_bounds=..., lat_bounds=..., num_points=...`.'
#     ## input case 1:
#     if ((lons is not None) and (lats is not None)):
#         ## eliminate case 2:
#         if ((lon_bounds is not None) or (lat_bounds is not None) or (grid_spacing is not None) or (num_points is not None)):
#             raise ValueError(error_msg)
#         ## execution:
#         pass
#     ## input case 2:
#     elif ((lon_bounds is not None) and (lat_bounds is not None)):
#         ## eliminate case 1:
#         if ((lons is not None) or (lats is not None)):
#             raise ValueError(error_msg)
#         ## execution (based on `grid_spacing` xor `num_points`):
#         if ((grid_spacing is not None) and (num_points is None)):
#             lons = np.arange(lon_bounds[0], lon_bounds[1]+grid_spacing, grid_spacing)
#             lats = np.arange(lat_bounds[0], lat_bounds[1]+grid_spacing, grid_spacing)
#         elif ((grid_spacing is None) and (num_points is not None)):
#             lons = np.linspace(lon_bounds[0], lon_bounds[1], num_points)
#             lats = np.linspace(lat_bounds[0], lat_bounds[1], num_points)
#         else:
#             raise ValueError(error_msg + ' (HINT: Specify either `grid_spacing` OR `num_points`, but not both.)')
            


#     '''checks'''
#     _initialize()

#     if np.any(lons < -180) or np.any(lons > 360):
#         raise ValueError(f'One value in given `lons` array is out of range [-180, 360].')
#     if np.any(lats < -90) or np.any(lats > 90):
#         raise ValueError(f'One value in given `lats` array is out of range [-90, 90].')
    
#     lons = np.round(lons, 10)
#     lats = np.round(lats, 10)
    
#     lons = utils.plon2slon(lons) # this only modifies values btwn 180 and 360



#     '''accessing'''
#     if interpolate:
#         method = 'linear'
#     else:
#         method = 'nearest'

#     match quantity:

#         case 'topo' | 'moho':
#             arr = dat_moho_xr.interp(lon=lons, lat=lats, assume_sorted=True, method=method)[quantity]
        
#         case 'crust' | 'crustal thickness' | 'crthick':
#             interped = dat_moho_xr.interp(lon=lons, lat=lats, assume_sorted=True, method=method)
#             arr = (interped.topo - interped.moho)
        
#         case 'rho' | 'density' | 'crustal density':
#             vec_is_above_dichotomy = np.vectorize(is_above_dichotomy)
#             arr = vec_is_above_dichotomy(np.meshgrid(lons, lats)[0], np.meshgrid(lons, lats)[1])
#             arr = np.where(arr, get_model_info()['rho_north'], get_model_info()['rho_south'])
        
#         case _:
#             raise Exception('Invalid quantity. Options are ["topo", "moho", "crust"/"crustal thickness"/"crthick", "rho"/"density"/"crustal density"].')


#     if as_xarray == False:
#         arr = arr.values

#     return arr



---
---
## moho registry (switch to load once)

In [ ]:
# _df_moho_shcoeffs_registry : pd.DataFrame = ... 

# ## download a pre-computed registry of moho models, which provides a google drive download link and a sha256 hash for a given model name
# with utils.disable_pooch_logger():
#     fpath_moho_shcoeffs_registry = pooch.retrieve(
#         fname      = 'Crust_mohoSHcoeffs_rawdata_registry.json',
#         url        = r'https://drive.google.com/file/d/17JJuTFKkHh651-rt2J2eFKnxiki0w4ue/view?usp=sharing',
#         known_hash = 'sha256:1800ee2883dc6bcc82bd34eb2eebced5b59fbe6c593cbc4e9122271fd01c1491',
#         path       = dirpath_data_root / 'Crust' / 'moho', 
#         downloader = utils.download_gdrive_file,
#     )
# fpath_moho_shcoeffs_registry = Path(fpath_moho_shcoeffs_registry)

# ## load to pandas dataframe and split 'model_name' into components
# _df_moho_shcoeffs_registry = pd.read_json(fpath_moho_shcoeffs_registry, orient='index').reset_index()
# _df_moho_shcoeffs_registry.rename(columns={'index': 'model_name'}, inplace=True)
# _df_moho_shcoeffs_registry[['RIM', 'insight_thickness', 'rho_south', 'rho_north']] = _df_moho_shcoeffs_registry['model_name'].str.split('-', expand=True)
# _df_moho_shcoeffs_registry['insight_thickness'] = _df_moho_shcoeffs_registry['insight_thickness'].astype(int)
# _df_moho_shcoeffs_registry['rho_south'] = _df_moho_shcoeffs_registry['rho_south'].astype(int)
# _df_moho_shcoeffs_registry['rho_north'] = _df_moho_shcoeffs_registry['rho_north'].astype(int)

# _df_moho_shcoeffs_registry